In [1]:
import pandas as pd
import numpy as np
import re
from thefuzz import fuzz,process
import spacy


In [2]:
df = pd.read_csv('Data_withoutBillNo.csv')

C:\Users\Phattaradanai\AppData\Local\Temp\ipykernel_20752\254014157.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data_withoutBillNo.csv')


In [8]:
df.head()

,BILL_INDEX,BILL_DATE,CONSIGNEE_TAMBON,CONSIGNEE_AMPHUR,CONSIGNEE_PROVINCE,CONSIGNEE_ZIPCODE,ITEM_CODE,ITEM_DESCแสดงหน้าบิล,ITEM_NAMEในระบบ,QTY,UNIT_NAME,REG_Code,DEST_CODE
0,0,20240301,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,เอกสาร ...,1,ซอง ...,R20,D06
1,1,20240301,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร ...,เอกสาร ...,1,ซอง ...,R20,D06
2,2,20240301,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร ...,เอกสาร ...,1,ซอง ...,R20,D06
3,3,20240301,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารใบสมัครงาน ...,เอกสาร ...,1,ซอง ...,R20,D06
4,4,20240301,ป่าสัก,เมืองลำพูน,ลำพูน,51000,10601.229,สินค้าตกค้าง/กันกระแทก ...,สินค้าตกค้าง ...,14,ห่อ ...,R20,D33


In [2]:
df = pd.read_csv('Data_withoutBillNo.csv')

# Reset_index
df = df.drop(columns=['BILL_INDEX'])
# Convert column to date_time
df['BILL_DATE'] = pd.to_datetime(df['BILL_DATE'], format='%Y%m%d')
# Rename column
df = df.rename(columns={
    'ITEM_DESCแสดงหน้าบิล':'ITEM_DESC',
    'ITEM_NAMEในระบบ'    :'ITEM_NAME',
    'CONSIGNEE_TAMBON' : 'TAMBON',
    'CONSIGNEE_AMPHUR' : 'AMPHUR',
    'CONSIGNEE_PROVINCE' : 'PROVINCE',
    'CONSIGNEE_ZIPCODE' : 'ZIPCODE',
    'REG_Code' : 'Sender',
    'DEST_CODE' : 'Receiver'
    })
df['ITEM_DESC'] = df['ITEM_DESC'].str.strip()
df['ITEM_NAME'] = df['ITEM_NAME'].str.strip()
df['UNIT_NAME'] = df['UNIT_NAME'].str.strip()


C:\Users\Phattaradanai\AppData\Local\Temp\ipykernel_24060\300080690.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data_withoutBillNo.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184926 entries, 0 to 1184925
Data columns (total 12 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   BILL_DATE  1184926 non-null  datetime64[ns]
 1   TAMBON     856506 non-null   object        
 2   AMPHUR     974473 non-null   object        
 3   PROVINCE   1184924 non-null  object        
 4   ZIPCODE    1184926 non-null  object        
 5   ITEM_CODE  1184926 non-null  object        
 6   ITEM_DESC  1184926 non-null  object        
 7   ITEM_NAME  1184926 non-null  object        
 8   QTY        1184926 non-null  object        
 9   UNIT_NAME  1184926 non-null  object        
 10  Sender     1184926 non-null  object        
 11  Receiver   1184925 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 108.5+ MB


## Clean Text

In [4]:
df_2 = df[df['ITEM_DESC']==df['ITEM_NAME']]
df_3 = df[~(df['ITEM_DESC']==df['ITEM_NAME'])]

In [5]:
# delete_row
df_3 = df_3[df_3['ITEM_NAME'] != '1']
df_3[df_3['ITEM_NAME']=='ยาวกระบะ']['ITEM_NAME'] = 'ยาง'
df_3[df_3['ITEM_NAME'].str.startswith('ผลไม้')] ='ผลไม้'
df_3[df_3['ITEM_NAME'].str.startswith('พริก')] ='พริก'
df_3 = df_3[df_3['ITEM_NAME'] != '18']
df_3 = df_3[df_3['ITEM_NAME'] != 'zhk']

C:\Users\Phattaradanai\AppData\Local\Temp\ipykernel_24060\3721225943.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'ผลไม้' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_3[df_3['ITEM_NAME'].str.startswith('ผลไม้')] ='ผลไม้'


In [6]:
df_3['ITEM_NAME'] = df_3['ITEM_NAME'].replace(r'\d+', '', regex=True).str.strip()
df_3[df_3['ITEM_NAME'].str.startswith('อาหาร')] ='อาหาร'
df_3['ITEM_NAME'] = df_3['ITEM_NAME'].replace(r"\s*\(.*?\)", '', regex=True).str.strip()
df_3['ITEM_NAME'] = df_3['ITEM_NAME'].replace(r'\(', '', regex=True).str.strip()
df_3['ITEM_NAME'] = df_3['ITEM_NAME'].replace(r'\*', '', regex=True).str.strip()

In [7]:
list_ITEM_NAME = ['สินค้าทั่วไป','ยางนอก','ค่าบรรทุก','เหมา','จักรยาน','กระเทียม'
                  ,'ดอกไม้','ชิ้นงาน','น้ำมันพืช','กระสอบ','อุปกรณ์','ไส้','สินค้าตกค้าง',
                  'ผักสดแช่เย็น','ของสด','มะเขือเทศ','จดหมายฟรี','กระเบื้อง','ไม้อัด','ยาง','ซองเอกสาร',
                  'เอกสาร','ตาข่าย','ส่งของภายใน','ผลไม้','กล่องบริษัท','ต้นหอม','หอมแดง','ฟักทอง','ชอกโกแลตและขนม',
                  'ถัง','ถั่ว','ถังไฟเบอร์','มันฝรั่ง','อะไหล่','พริก','ไซส์','สินค้าเสียหาย','ทราย','ยางกระบะ','ข่า/ตะไคร้','ผลไม้',
                  'พริก','ผ้าม่าน','ตะเกียบ','หอมหัวใหญ่','เสื้อ'
                 ]
for name in list_ITEM_NAME:
    match = process.extract(name, df_3['ITEM_NAME'], limit=df_3['ITEM_NAME'].shape[0])
    for potentail_match in match:
        if potentail_match[1]>=80:
            df_3.loc[df_3['ITEM_NAME']==potentail_match[0],'ITEM_NAME'] = name
df_3

,BILL_DATE,TAMBON,AMPHUR,PROVINCE,ZIPCODE,ITEM_CODE,ITEM_DESC,ITEM_NAME,QTY,UNIT_NAME,Sender,Receiver
0,2024-03-01 00:00:00,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถานที่,เอกสาร,1,ซอง,R20,D06
3,2024-03-01 00:00:00,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารใบสมัครงาน,เอกสาร,1,ซอง,R20,D06
4,2024-03-01 00:00:00,ป่าสัก,เมืองลำพูน,ลำพูน,51000,10601.229,สินค้าตกค้าง/กันกระแทก,สินค้าตกค้าง,14,ห่อ,R20,D33
10,2024-03-01 00:00:00,ชมพู,เมืองลำปาง,ลำปาง,52100,10601.229,สินค้าตกค้าง/แกลลอน3ลิตร,สินค้าตกค้าง,150,ห่อ,R20,D32
11,2024-03-01 00:00:00,ชมพู,เมืองลำปาง,ลำปาง,52100,50101.002,ตาข่าย6622252,ตาข่าย,1,ผืน,R20,D32
...,...,...,...,...,...,...,...,...,...,...,...,...
1184823,2024-03-31 00:00:00,คลองหนึ่ง,คลองหลวง,ปทุมธานี,12120,99999.006,กระเทียม 36x20x10cm / 10kgกลีบ,กระเทียม,134,ถุง,R30,D07
1184864,2024-03-31 00:00:00,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,20201.023,เอกสารบิลเงินสดสาขาเชียงดาว,เอกสาร,1,ซอง,R04,D06
1184865,2024-03-31 00:00:00,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,20201.023,เอกสารบิลขนส่งสาขาเชียงดาว,เอกสาร,1,กล่อง,R04,D06
1184866,2024-03-31 00:00:00,สี่แยกมหานาค,ดุสิต,กรุงเทพมหานคร,10300,10602.009,เอกสาร**ลังมีล้อ,เอกสาร,1,ใบ,R06,D37


In [8]:
df_3[~(df_3['ITEM_NAME'].isin(list_ITEM_NAME))]['ITEM_NAME'].value_counts()

ITEM_NAME
อาหาร        360
เชือก        291
ข้าวสาร       82
ขนมส่งคืน     77
ยาสัตว์       76
            ... 
หัวเร่งว       1
สวายไฟ         1
หน้าโ๊ะ        1
ซฺป            1
กา             1
Name: count, Length: 2263, dtype: int64

## Create Zone

In [9]:
df['ZIPCODE'] = df['ZIPCODE'].str.strip()

In [10]:
zone_mapping = [
    (r'^1[0-8]\d{3}$', 'Central zone'),
    (r'^2[0-7]\d{3}$', 'Eastern zone'),
    (r'^3[0-9]\d{3}$', 'Northeastern part 1'),
    (r'^4[0-9]\d{3}$', 'Northeastern part 2'),
    (r'^5[0-8]\d{3}$', 'Upper Northern zone'),
    (r'^6[0-7]\d{3}$', 'Lower Northern zone'),
    (r'^7[0-7]\d{3}$', 'Lower Central zone'),
    (r'^8[0-6]\d{3}$', 'Southern zone'),
    (r'^9[0-6]\d{3}$', 'Southern border zone'),
]

In [11]:
def determine_zone(zipcode):
    for pattern, zone in zone_mapping:
        if re.match(pattern, zipcode):
            return zone
    return 'Unknown'

In [12]:
df['Zone'] = df['ZIPCODE'].apply(determine_zone)

## Drop Unwanted Column

In [13]:
df.drop(['Sender','Receiver'],axis=1)

,BILL_DATE,TAMBON,AMPHUR,PROVINCE,ZIPCODE,ITEM_CODE,ITEM_DESC,ITEM_NAME,QTY,UNIT_NAME,Zone
0,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถานที่,เอกสาร,1,ซอง,Upper Northern zone
1,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,Upper Northern zone
2,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,Upper Northern zone
3,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารใบสมัครงาน,เอกสาร,1,ซอง,Upper Northern zone
4,2024-03-01,ป่าสัก,เมืองลำพูน,ลำพูน,51000,10601.229,สินค้าตกค้าง/กันกระแทก,สินค้าตกค้าง,14,ห่อ,Upper Northern zone
...,...,...,...,...,...,...,...,...,...,...,...
1184921,2024-03-31,ช้างคลาน,เมืองเชียงใหม่,เชียงใหม่,,99999.002,ของสด,ของสด,1,ฟญ,Unknown
1184922,2024-03-31,หนองผึ้ง,สารภี,เชียงใหม่,50140,99999.002,หมุแผ่น,หมุแผ่น,1,กล่อง,Upper Northern zone
1184923,2024-03-31,ช้างม่อย,เมืองเชียงใหม่,เชียงใหม่,50300,99999.002,ของสด,ของสด,2,ถัง,Upper Northern zone
1184924,2024-03-31,NaN,NaN,เชียงใหม่,,99999.005,กาแฟสาร,กาแฟสาร,2,กระสอบ,Unknown


In [14]:
df['ITEM_DESC'].value_counts()

ITEM_DESC
ค่าขนส่งส่วนเพิ่ม                    104845
ผ้า                                   71273
ดอกไม้สด กล่องใหญ่ (46x100x60 cm)     42511
อุปกรณ์                               40198
อะไหล่                                34389
                                      ...  
โบว์เวอร์พลาสสติก                         1
เครื่องปั่นSW                             1
ผลไมัอบแห้ง                               1
พุทราไร้เมล็ดใหญ่                         1
ของสด หนังปลา                             1
Name: count, Length: 94803, dtype: int64

## Create Sample

In [15]:
df_sample = df.sample(frac=0.1)

In [16]:
df_sample.to_csv('sample.csv',index=False)

## Save cleaned file to CSV

In [50]:
df.to_csv('cleaned_text.csv',index=False)

## Load data from cleaned_text.csv (for save time)

In [3]:
df_cleaned_text = pd.read_csv('cleaned_text.csv')
df_cleaned_text['BILL_DATE'] = pd.to_datetime(df_cleaned_text['BILL_DATE'])

C:\Users\Phattaradanai\AppData\Local\Temp\ipykernel_26200\586816814.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned_text = pd.read_csv('cleaned_text.csv')


In [52]:
df_cleaned_text.head()

,BILL_DATE,TAMBON,AMPHUR,PROVINCE,ZIPCODE,ITEM_CODE,ITEM_DESC,ITEM_NAME,QTY,UNIT_NAME,Sender,Receiver,Zone,Month,Day_Name
0,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถานที่,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
1,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
2,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
3,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารใบสมัครงาน,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
4,2024-03-01,ป่าสัก,เมืองลำพูน,ลำพูน,51000,10601.229,สินค้าตกค้าง/กันกระแทก,สินค้าตกค้าง,14,ห่อ,R20,D33,Upper Northern zone,3,Friday


In [5]:
df_cleaned_text['Zone'].value_counts()

Zone
Unknown                 894714
Upper Northern zone     221325
Central zone             61449
Lower Central zone        4654
Lower Northern zone        898
Northeastern part 1        565
Northeastern part 2        507
Southern border zone       380
Eastern zone               242
Southern zone              192
Name: count, dtype: int64

In [16]:
df_cleaned_text['ZIPCODE'].isnull().sum()

892523

In [7]:
df_cleaned_text[df_cleaned_text['Zone'] == 'Unknown']

,BILL_DATE,TAMBON,AMPHUR,PROVINCE,ZIPCODE,ITEM_CODE,ITEM_DESC,ITEM_NAME,QTY,UNIT_NAME,Sender,Receiver,Zone,Month,Day_Name
73,2024-03-01,วัดเกต,เมืองเชียงใหม่,เชียงใหม่,NaN,99999.000,พลาสติก,พลาสติก,1,มัด,R19,D06,Unknown,3,Friday
94,2024-03-01,บ้านกลาง,เมืองลำพูน,ลำพูน,NaN,99999.000,ถุงพลาสติก,ถุงพลาสติก,13,ห่อ,R26,D33,Unknown,3,Friday
95,2024-03-01,บ้านกลาง,เมืองลำพูน,ลำพูน,NaN,99999.000,ม้วนพลาสติก,ม้วนพลาสติก,10,ม้วน,R26,D33,Unknown,3,Friday
96,2024-03-01,บ้านกลาง,เมืองลำพูน,ลำพูน,NaN,99999.000,ถังเหล็กแบตเตอรี่โฟคลิฟท์,ถังเหล็กแบตเตอรี่โฟคลิฟท์,2,ตัว,R26,D33,Unknown,3,Friday
97,2024-03-01,วังพร้าว,เกาะคา,ลำปาง,NaN,99999.000,กระดาษทราย,กระดาษทราย,1,กล่อง,R26,D32,Unknown,3,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184919,2024-03-31,NaN,NaN,เชียงใหม่,NaN,99999.002,กล้วยไม้,กล้วยไม้,5,กล่อง,R22,D06,Unknown,3,Sunday
1184920,2024-03-31,ป่าตัน,เมืองเชียงใหม่,เชียงใหม่,NaN,99999.002,ของสด,ของสด,1,ฟล,R22,D06,Unknown,3,Sunday
1184921,2024-03-31,ช้างคลาน,เมืองเชียงใหม่,เชียงใหม่,NaN,99999.002,ของสด,ของสด,1,ฟญ,R22,D06,Unknown,3,Sunday
1184924,2024-03-31,NaN,NaN,เชียงใหม่,NaN,99999.005,กาแฟสาร,กาแฟสาร,2,กระสอบ,R05,D06,Unknown,3,Sunday


## Pivot Item Name

In [4]:
pivot_item_name = df_cleaned_text.pivot_table(index='ITEM_NAME', aggfunc='size').reset_index(name='Count')

In [5]:
pivot_item_name = pivot_item_name.sort_values(by='Count', ascending=False)

In [6]:
pivot_item_name.head(20)

,ITEM_NAME,Count
10498,ค่าขนส่งส่วนเพิ่ม,104845
27363,ผ้า,71174
13852,ดอกไม้สด กล่องใหญ่ (46x100x60 cm),42583
57351,อุปกรณ์,40223
55782,อะไหล่,34374
66972,เสื้อผ้า,32886
44514,รองเท้า,15644
62480,เครื่องเขียน,12966
43408,ยาพืช,9485
69624,แตงกวา 29x43x37 cm / 20 kg,8929


In [56]:
pivot_item_name.to_csv('item_name.csv',index=False)

## Create Month and Day_Name Column

In [7]:
df_cleaned_text['Month'] = df_cleaned_text['BILL_DATE'].dt.month

In [8]:
df_cleaned_text['Day_Name'] = df_cleaned_text['BILL_DATE'].dt.day_name()

In [9]:
df_cleaned_text.head()

,BILL_DATE,TAMBON,AMPHUR,PROVINCE,ZIPCODE,ITEM_CODE,ITEM_DESC,ITEM_NAME,QTY,UNIT_NAME,Sender,Receiver,Zone,Month,Day_Name
0,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถานที่,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
1,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
2,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสาร,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
3,2024-03-01,ฟ้าฮ่าม,เมืองเชียงใหม่,เชียงใหม่,50000,10105.005,เอกสารใบสมัครงาน,เอกสาร,1,ซอง,R20,D06,Upper Northern zone,3,Friday
4,2024-03-01,ป่าสัก,เมืองลำพูน,ลำพูน,51000,10601.229,สินค้าตกค้าง/กันกระแทก,สินค้าตกค้าง,14,ห่อ,R20,D33,Upper Northern zone,3,Friday


In [60]:
month_df = df_cleaned_text.groupby(['Month']).size().reset_index(name='Counts')

In [61]:
month_df.head()

,Month,Counts
0,1,208756
1,2,191138
2,3,213971
3,4,175888
4,5,205455


In [62]:
day_name_df = df_cleaned_text.groupby(['Day_Name']).size().reset_index(name='Counts')

In [63]:
day_name_df

,Day_Name,Counts
0,Friday,193367
1,Monday,192180
2,Saturday,164290
3,Sunday,26653
4,Thursday,205311
5,Tuesday,204085
6,Wednesday,199040


In [64]:
month_df.to_csv('month.csv',index=False)

In [65]:
day_name_df.to_csv('day_name.csv',index=False)

## Reduce Unknow zone

In [21]:
df_cleaned_text['Zone'].value_counts()

In [5]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'เชียงใหม่', 'Zone'] = 'Upper Northern zone'

In [6]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'เชียงราย', 'Zone'] = 'Upper Northern zone'

In [7]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'แพร่', 'Zone'] = 'Upper Northern zone'

In [8]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'พะเยา', 'Zone'] = 'Upper Northern zone'

In [9]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'แม่ฮ่องสอน', 'Zone'] = 'Upper Northern zone'

In [11]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'ลำปาง', 'Zone'] = 'Upper Northern zone'

In [13]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'ลำพูน', 'Zone'] = 'Upper Northern zone'

In [14]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'น่าน', 'Zone'] = 'Upper Northern zone'

In [15]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'กรุงเทพมหานคร', 'Zone'] = 'Central zone'

In [16]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'ปทุมธานี', 'Zone'] = 'Central zone'

In [17]:
df_cleaned_text.loc[df_cleaned_text['PROVINCE'] == 'ราชบุรี', 'Zone'] = 'Eastern zone'

In [18]:
df_cleaned_text['Zone'].value_counts()

Zone
Upper Northern zone     903888
Central zone            244022
Eastern zone             16296
Unknown                  13783
Lower Central zone        4418
Lower Northern zone        884
Northeastern part 1        563
Northeastern part 2        502
Southern border zone       379
Southern zone              191
Name: count, dtype: int64

In [19]:
df_cleaned_text.to_csv('aftercleanedzone.csv',index=False)